In [1]:
import os
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


## Download dataset 
https://drive.google.com/drive/u/3/folders/1sHh6NvuKX6RB5OytLwf4kaqfQ9svJNDQ

## Load data

In [2]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")

x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [3]:
# It's a multi-class classification problem 
class_index = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4,
               'dog': 5, 'frog': 6,'horse': 7,'ship': 8, 'truck': 9}
print(np.unique(y_train))

[0 1 2 3 4 5 6 7 8 9]


![image](https://img-blog.csdnimg.cn/20190623084800880.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3lqcDE5ODcxMDEz,size_16,color_FFFFFF,t_70)

## Data preprocess

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to one-hot encoding (keras model requires one-hot label as inputs)
num_classes = 10
print(y_train[0])
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train[0])

[9]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


## Build model & training (Keras)

In [5]:
# # Builde model
# model = Sequential() # Sequential groups a linear stack of layers
# model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=x_train.shape[1:])) # Add Convolution layers
# model.add(Activation('relu')) # Add Relu activation for non-linearity
# model.add(Conv2D(filters=32, kernel_size=(3, 3))) # Add Convolution layers
# model.add(Activation('relu')) # Add Relu activation for non-linearity
# model.add(MaxPooling2D(pool_size=(4, 4))) # Add Max pooling to lower the sptail dimension

# model.add(Flatten()) # Flatten the featuremaps
# model.add(Dense(units=512)) # Add dense layer with 512 neurons
# model.add(Activation('relu')) # Add Relu activation for non-linearity
# model.add(Dropout(0.25))
# model.add(Dense(units=num_classes)) # Add final output layer for 10 classes
# model.add(Activation('softmax')) # Add softmax activation to transfer logits into probabilities

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())  # Flatten the featuremaps
model.add(Dense(units=512))  # Add dense layer with 512 neurons
model.add(Activation('relu'))  # Add Relu activation for non-linearity
model.add(Dense(units=num_classes))  # Add final output layer for 10 classes
model.add(Activation('softmax'))  # Add softmax activation to transfer logits into probabilities

# data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    shear_range=0.1,
    zoom_range=0.2,
    fill_mode='nearest'
    )
datagen.fit(x_train)

# initiate SGD optimizer
# opt = keras.optimizers.SGD()
opt = keras.optimizers.Adam(lr=0.0002)

# Compile the model with loss function and optimizer, and evaluate with accuracy
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Setup some hyperparameters
batch_size = 32
epochs = 50

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs, validation_data=(x_test, y_test), shuffle=True)
# Fit the data into model
# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_data=(x_test, y_test),
#           shuffle=True)


Epoch 1/50
1563/1563 [==============================] - 296s 190ms/step - loss: 1.5948 - accuracy: 0.4302 - val_loss: 1.2423 - val_accuracy: 0.5543
Epoch 2/50
1563/1563 [==============================] - 290s 185ms/step - loss: 1.2570 - accuracy: 0.5504 - val_loss: 1.3632 - val_accuracy: 0.5508
Epoch 3/50
1563/1563 [==============================] - 288s 184ms/step - loss: 1.1018 - accuracy: 0.6095 - val_loss: 0.9436 - val_accuracy: 0.6578
Epoch 4/50
1563/1563 [==============================] - 283s 181ms/step - loss: 0.9882 - accuracy: 0.6513 - val_loss: 0.8527 - val_accuracy: 0.7034
Epoch 5/50
1563/1563 [==============================] - 294s 188ms/step - loss: 0.9115 - accuracy: 0.6795 - val_loss: 0.8260 - val_accuracy: 0.7118
Epoch 6/50
1563/1563 [==============================] - 287s 183ms/step - loss: 0.8470 - accuracy: 0.7027 - val_loss: 0.8054 - val_accuracy: 0.7290
Epoch 7/50
1563/1563 [==============================] - 294s 188ms/step - loss: 0.8061 - accuracy: 0.7183 - val_

In [6]:
y_pred = model.predict(x_test)
print(y_pred.shape) # 10000 samples, each sample with probaility of 10 classes

(10000, 10)


In [7]:
y_pred[0] 

array([1.2368728e-05, 9.9895489e-01, 4.3988424e-12, 1.1496225e-09,
       8.7085547e-12, 6.7420852e-10, 7.1641616e-11, 3.5065888e-08,
       3.0515128e-09, 1.0327406e-03], dtype=float32)

In [8]:
np.argmax(y_pred[0]) # argmax to find the predict class with highest probability. 9=truck

1

In [9]:
y_pred = np.argmax(y_pred, axis=1)

## DO NOT MODIFY CODE BELOW!
**Please screen shot your results and post it on your report**

In [ ]:
y_pred = your_model.predict(x_test)

In [10]:
assert y_pred.shape == (10000,)

In [11]:
y_test = np.load("y_test.npy")
print("Accuracy of my model on test set: ", accuracy_score(y_test, y_pred))

Accuracy of my model on test set:  0.8705
